In [ ]:
QueryNodes="""
    PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX c:  <http://iec.ch/TC57/CIM100#>
    SELECT DISTINCT ?busname ?cnid ?tpnid  WHERE {
      VALUES ?fdrid {"%s"}
    ?fdr c:IdentifiedObject.mRID ?fdrid.
    ?bus c:ConnectivityNode.ConnectivityNodeContainer ?fdr.
    ?bus c:ConnectivityNode.TopologicalNode ?tp.
    ?bus r:type c:ConnectivityNode.
    ?bus c:IdentifiedObject.name ?busname.
    ?bus c:IdentifiedObject.mRID ?cnid.
    ?fdr c:IdentifiedObject.name ?feeder.
    ?trm c:Terminal.ConnectivityNode ?bus.
    ?trm c:Terminal.ConductingEquipment ?ce.

    OPTIONAL {
    ?ce  c:ConductingEquipment.BaseVoltage ?bv.
    ?bv  c:BaseVoltage.nominalVoltage ?nomv.
      }
    bind(strafter(str(?tp), str("http://localhost:8889/bigdata/namespace/kb/sparql#")) as ?tpnid)
    }
    GROUP by ?busname ?cnid ?tpnid
    ORDER by ?busname
    """%model_mrid

results = gapps.query_data(query = QueryNodes, timeout = 60)
Node_query = results['data']['results']['bindings']

In [ ]:
ConnNodeEquipDict = {}

for i0 in range(len(Node_query)):
    node=Node_query[i0]['cnid']['value']

    ConnNodeEquipDict[node] = {}
    ConnNodeEquipDict[node]['name'] = Node_query[i0]['busname']['value']
    ConnNodeEquipDict[node]['tpid'] = Node_query[i0]['tpnid']['value']
    if 'nomv' in Node_query[i0]: 
        ConnNodeEquipDict[node]['nomv'] = Node_query[i0]['nomv']['value']
    else:
        ConnNodeEquipDict[node]['nomv'] = []
    
    ConnNodeEquipDict[node]['ACLineSegment.mRID'] = []
    ConnNodeEquipDict[node]['TransformerEnd.mRID'] = []
    ConnNodeEquipDict[node]['switch.mRID'] = []
    ConnNodeEquipDict[node]['SynchronousMachine.mRID'] = []
    ConnNodeEquipDict[node]['EnergyConsumer.mRID'] = []
    ConnNodeEquipDict[node]['PowerElectronicsConnection.mRID'] = []
    ConnNodeEquipDict[node]['measurements'] = []
    

In [ ]:
QueryLines="""
    PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX c:  <http://iec.ch/TC57/CIM100#>
    SELECT ?name ?bus1 ?bus2 ?id ?tname1 ?term1 ?tname2 ?term2 ?node1 ?node2 ?tpnode1 ?tpnode2 (group_concat(distinct ?phs;separator="") as ?phases) WHERE {
    SELECT ?name ?bus1 ?bus2 ?phs ?id ?tname1 ?term1 ?tname2 ?term2 ?node1 ?node2 ?tpnode1 ?tpnode2 WHERE {
    VALUES ?fdrid {"%s"}  
    ?fdr c:IdentifiedObject.mRID ?fdrid.
    ?s r:type c:ACLineSegment.
    ?s c:Equipment.EquipmentContainer ?fdr.
    ?s c:IdentifiedObject.name ?name.
    ?s c:IdentifiedObject.mRID ?id.
    ?t1 c:Terminal.ConductingEquipment ?s.
    ?t1 c:ACDCTerminal.sequenceNumber "1".
    ?t1 c:Terminal.ConnectivityNode ?cn1. 
    ?t1 c:IdentifiedObject.name ?tname1.
    ?cn1 c:IdentifiedObject.name ?bus1.
    ?cn1 c:ConnectivityNode.TopologicalNode ?tp1.
    ?t2 c:Terminal.ConductingEquipment ?s.
    ?t2 c:ACDCTerminal.sequenceNumber "2".
    ?t2 c:Terminal.ConnectivityNode ?cn2. 
    ?t2 c:IdentifiedObject.name ?tname2.
    ?cn2 c:ConnectivityNode.TopologicalNode ?tp2.
    ?cn2 c:IdentifiedObject.name ?bus2.
    bind(strafter(str(?t),"#") as ?tid).
        bind(strafter(str(?t1), "#") as ?term1) 
        bind(strafter(str(?t2), "#") as ?term2)
        bind(strafter(str(?cn1), "#") as ?node1)
        bind(strafter(str(?cn2), "#") as ?node2)
        bind(strafter(str(?tp1), "#") as ?tpnode1)
        bind(strafter(str(?tp2), "#") as ?tpnode2)
            OPTIONAL {?acp c:ACLineSegmentPhase.ACLineSegment ?s.
            ?acp c:ACLineSegmentPhase.phase ?phsraw.
            bind(strafter(str(?phsraw),"SinglePhaseKind.") as ?phs) }

    } ORDER BY ?name ?phs
    }
    GROUP BY ?name ?bus1 ?bus2 ?id ?tname1 ?term1 ?tname2 ?term2 ?node1 ?node2 ?tpnode1 ?tpnode2
    ORDER BY ?name
    """%model_mrid
results = gapps.query_data(query = QueryLines, timeout = 60)
LineQuery = results['data']['results']['bindings']